In [19]:
# COMPLETE WORKING CROP RECOMMENDATION SYSTEM
# This code includes all required components and fixes the "Not Fitted" error

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

print("🔄 Creating Dataset and Training Model...")

# === STEP 1: CREATE SAMPLE DATASET ===
np.random.seed(42)

# Define crops and their agricultural requirements based on research data
crops_info = {
    'rice': {'N': 80, 'P': 40, 'K': 40, 'temp_range': (20, 35), 'ph_range': (5.5, 7.0), 'humidity_range': (80, 95), 'rainfall_range': (150, 300)},
    'wheat': {'N': 120, 'P': 60, 'K': 40, 'temp_range': (15, 25), 'ph_range': (6.0, 7.5), 'humidity_range': (50, 70), 'rainfall_range': (50, 100)},
    'maize': {'N': 150, 'P': 75, 'K': 75, 'temp_range': (20, 30), 'ph_range': (6.0, 7.0), 'humidity_range': (60, 80), 'rainfall_range': (60, 110)},
    'cotton': {'N': 120, 'P': 50, 'K': 50, 'temp_range': (21, 30), 'ph_range': (5.8, 8.0), 'humidity_range': (50, 80), 'rainfall_range': (50, 100)},
    'sugarcane': {'N': 200, 'P': 80, 'K': 150, 'temp_range': (20, 30), 'ph_range': (6.5, 7.5), 'humidity_range': (70, 85), 'rainfall_range': (75, 150)},
    'soybean': {'N': 80, 'P': 70, 'K': 70, 'temp_range': (20, 30), 'ph_range': (6.0, 7.0), 'humidity_range': (60, 80), 'rainfall_range': (45, 100)},
    'chickpea': {'N': 25, 'P': 60, 'K': 40, 'temp_range': (15, 25), 'ph_range': (6.2, 7.8), 'humidity_range': (50, 70), 'rainfall_range': (30, 40)},
    'groundnut': {'N': 25, 'P': 50, 'K': 75, 'temp_range': (20, 30), 'ph_range': (6.0, 7.0), 'humidity_range': (50, 75), 'rainfall_range': (50, 125)},
    'barley': {'N': 125, 'P': 60, 'K': 40, 'temp_range': (12, 25), 'ph_range': (6.0, 7.5), 'humidity_range': (55, 70), 'rainfall_range': (45, 65)},
    'banana': {'N': 300, 'P': 100, 'K': 300, 'temp_range': (26, 30), 'ph_range': (6.0, 7.5), 'humidity_range': (75, 85), 'rainfall_range': (100, 180)}
}

# Generate training data (2200 samples total)
data = []
for crop, info in crops_info.items():
    for _ in range(220):  # 220 samples per crop
        sample = {
            'N': max(0, np.random.normal(info['N'], 15)),
            'P': max(0, np.random.normal(info['P'], 10)),
            'K': max(0, np.random.normal(info['K'], 10)),
            'temperature': np.random.uniform(info['temp_range'][0], info['temp_range'][1]),
            'humidity': np.random.uniform(info['humidity_range'][0], info['humidity_range'][1]),
            'ph': np.random.uniform(info['ph_range'][0], info['ph_range'][1]),
            'rainfall': np.random.uniform(info['rainfall_range'][0], info['rainfall_range'][1]),
            'label': crop
        }
        data.append(sample)

df = pd.DataFrame(data)

# === STEP 2: TRAIN THE MODEL ===
feature_names = ['N', 'P', 'K', 'temperature', 'humidity', 'ph', 'rainfall']
X = df[feature_names]
y = df['label']

# Encode labels
le = LabelEncoder()
y_encoded = le.fit_transform(y)

# Split and train the model
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)  # ✅ THIS IS THE CRUCIAL STEP THAT WAS MISSING

# Create average NPK values for fertilizer recommendations
avg_npk = df.groupby('label')[['N', 'P', 'K']].mean()

print("✅ Model Training Completed Successfully!")
print(f"Model Accuracy: {model.score(X_test, y_test):.4f}")

# === STEP 3: CREATE WIDGETS WITH BETTER STYLING ===
n_slider = widgets.IntSlider(description='N (kg/ha)', value=50, min=0, max=300, style={'description_width': 'initial'})
p_slider = widgets.IntSlider(description='P (kg/ha)', value=50, min=0, max=150, style={'description_width': 'initial'})
k_slider = widgets.IntSlider(description='K (kg/ha)', value=50, min=0, max=300, style={'description_width': 'initial'})
temp_slider = widgets.FloatSlider(description='Temperature (°C)', value=25.0, min=10, max=40, style={'description_width': 'initial'})
humidity_slider = widgets.FloatSlider(description='Humidity (%)', value=70.0, min=30, max=100, style={'description_width': 'initial'})
ph_slider = widgets.FloatSlider(description='pH', value=6.5, min=4.0, max=9.0, step=0.1, style={'description_width': 'initial'})
rainfall_slider = widgets.FloatSlider(description='Rainfall (mm)', value=100.0, min=20, max=350, style={'description_width': 'initial'})

# Soil type dropdown
soil_types = ['Sandy', 'Loamy', 'Clay', 'Silty', 'Peaty', 'Chalky']
soil_dropdown = widgets.Dropdown(options=soil_types, description='Soil Type:', style={'description_width': 'initial'})

# Soil-crop suitability mapping
soil_crop_suitability = {
    'Sandy': ['maize', 'cotton', 'groundnut'],
    'Loamy': ['rice', 'wheat', 'sugarcane', 'banana'],
    'Clay': ['rice', 'soybean', 'cotton'],
    'Silty': ['rice', 'wheat', 'barley'],
    'Peaty': ['rice', 'sugarcane'],
    'Chalky': ['barley', 'maize']
}

def filter_crops_by_soil(predicted_crop, soil):
    """Filter crops based on soil suitability"""
    suitable = soil_crop_suitability.get(soil, [])
    if predicted_crop in suitable:
        return predicted_crop
    return suitable[0] if suitable else predicted_crop

def fertilizer_quantity_advisory(crop, N, P, K):
    """Generate fertilizer recommendations based on crop requirements"""
    try:
        if crop in avg_npk.index:
            recommended = avg_npk.loc[crop]
            advice = []

            n_diff = recommended['N'] - N
            p_diff = recommended['P'] - P
            k_diff = recommended['K'] - K

            advice.append(f"Nitrogen: {'Add' if n_diff > 0 else 'Reduce'} {abs(n_diff):.1f} kg/ha")
            advice.append(f"Phosphorus: {'Add' if p_diff > 0 else 'Reduce'} {abs(p_diff):.1f} kg/ha")
            advice.append(f"Potassium: {'Add' if k_diff > 0 else 'Reduce'} {abs(k_diff):.1f} kg/ha")

            return advice
        else:
            return ["No specific recommendations available for this crop"]
    except Exception as e:
        return [f"Error generating recommendations: {str(e)}"]

# Output widgets
output_crop = widgets.Output()
output_fert = widgets.Output()
output_viz = widgets.Output()

# === STEP 4: IMPROVED BUTTON CALLBACK WITH ERROR HANDLING ===
def on_button_click(b):
    """Button click handler with comprehensive error handling"""

    # Clear previous outputs
    with output_crop:
        output_crop.clear_output()
        try:
            # Gather user inputs
            user_data = pd.DataFrame([[
                n_slider.value, p_slider.value, k_slider.value,
                temp_slider.value, humidity_slider.value,
                ph_slider.value, rainfall_slider.value
            ]], columns=feature_names)

            # Make prediction using the trained model
            pred_idx = model.predict(user_data)[0]
            crop_pred = le.inverse_transform([pred_idx])[0]

            # Adjust for soil type
            crop_final = filter_crops_by_soil(crop_pred, soil_dropdown.value)

            print(f"🌱 Recommended Crop (adjusted for {soil_dropdown.value} soil): {crop_final}")

        except Exception as e:
            print(f"❌ Error in crop prediction: {str(e)}")
            return

    # Fertilizer advisory
    with output_fert:
        output_fert.clear_output()
        try:
            if 'crop_final' in locals():
                advice = fertilizer_quantity_advisory(crop_final,
                                                    n_slider.value,
                                                    p_slider.value,
                                                    k_slider.value)
                print("💧 Fertilizer Advisory:")
                for line in advice:
                    print(f" - {line}")
            else:
                print("❌ Cannot generate fertilizer advice - crop prediction failed")
        except Exception as e:
            print(f"❌ Error in fertilizer advisory: {str(e)}")

    # Visualization
    with output_viz:
        output_viz.clear_output()
        try:
            if 'crop_final' in locals() and crop_final in avg_npk.index:
                avg_vals = avg_npk.loc[crop_final]
                nutrients = ['N', 'P', 'K']
                crop_values = [avg_vals[n] for n in nutrients]
                soil_values = [n_slider.value, p_slider.value, k_slider.value]

                fig, ax = plt.subplots(figsize=(8, 5))
                bar_width = 0.35
                index = np.arange(len(nutrients))

                ax.bar(index, crop_values, bar_width, label='Recommended Levels', color='#4caf50')
                ax.bar(index + bar_width, soil_values, bar_width, label='Your Soil', color='#ff9800')

                ax.set_xlabel('Nutrient')
                ax.set_ylabel('Quantity (kg/ha)')
                ax.set_title(f'Soil Nutrient Comparison for {crop_final.capitalize()}')
                ax.set_xticks(index + bar_width / 2)
                ax.set_xticklabels(nutrients)
                ax.legend()
                ax.grid(True, alpha=0.3)

                plt.tight_layout()
                plt.show()
            else:
                print("❌ Cannot generate visualization - insufficient data")
        except Exception as e:
            print(f"❌ Error in visualization: {str(e)}")

# === STEP 5: CREATE USER INTERFACE ===
button = widgets.Button(description='🔍 Recommend Crop', button_style='success')
button.on_click(on_button_click)

input_widgets = widgets.VBox([
    widgets.HTML("<h3>🌾 Crop Recommendation System</h3>"),
    n_slider, p_slider, k_slider,
    temp_slider, humidity_slider, ph_slider, rainfall_slider,
    soil_dropdown,
    button
])

output_widgets = widgets.VBox([
    widgets.HTML("<h4>📊 Results</h4>"),
    output_crop, output_fert, output_viz
])

# Display the application
display(widgets.HBox([input_widgets, output_widgets],
                     layout=widgets.Layout(justify_content='space-around')))

print("\n🎉 Crop Recommendation System Ready!")
print("🎯 The model is now properly trained and fitted")
print("🎯 All widgets are functional")
print("🎯 Error handling is implemented")

🔄 Creating Dataset and Training Model...
✅ Model Training Completed Successfully!
Model Accuracy: 0.9273



🎉 Crop Recommendation System Ready!
🎯 The model is now properly trained and fitted
🎯 All widgets are functional
🎯 Error handling is implemented


# New Section